# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [31]:
import ee
import geemap.foliumap as geemap
import webbrowser
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context
ee.Authenticate()
ee.Initialize(project='ee-juliaszymanska142')
# Define AOI here
aoi = ee.Geometry.Rectangle([19.884968, 50.012329, 19.908652, 50.025676])


## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [32]:
# Load image and calculate TB

def get_landsat_image(year):
    start_date = f'{year}-06-01'
    end_date = f'{year}-07-31'
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .sort('CLOUD_COVER')

    image = collection.first()
    if image is None:
        raise Exception(f'Brak obrazów Landsat 8 dla roku {year}')
    return image


image_2023 = get_landsat_image(2023)
thermal_band_2023 = image_2023.select('ST_B10')
TB_2023 = thermal_band_2023.multiply(0.00341802).add(149.0)


try:
    image_2013 = get_landsat_image(2013)
except Exception as e:
    print(f'⚠️ {e} - próbuję załadować 2014')
    image_2013 = get_landsat_image(2014)

thermal_band_2013 = image_2013.select('ST_B10')
TB_2013 = thermal_band_2013.multiply(0.00341802).add(149.0)




## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [33]:
# Load CORINE
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018').select('landcover').clip(aoi)


## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [34]:


# Create emissivity image

emissivity_dict = {
    111: 0.92,
    112: 0.92,
    121: 0.91,
    211: 0.96,
    311: 0.98,
    412: 0.97,
    324: 0.96,
    231: 0.97
}

corine_classes = list(emissivity_dict.keys())
emissivity_values = list(emissivity_dict.values())
emissivity = corine.remap(corine_classes, emissivity_values).rename('emissivity')
emissivity = emissivity.unmask(0.98)





## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [35]:
# Calculate LST
lambda_val = 10.8
c2 = 14388

lst_2023 = TB_2023.divide(
    ee.Image(1).add(
        (ee.Image(lambda_val).multiply(TB_2023))
        .divide(c2)
        .multiply(emissivity.log())
    )
).rename('LST')

lst_2013 = TB_2013.divide(
    ee.Image(1).add(
        (ee.Image(lambda_val).multiply(TB_2013))
        .divide(c2)
        .multiply(emissivity.log())
    )
).rename('LST')


## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [36]:
visualization_params = {
    'min': 290,
    'max': 325,
    'palette': ['blue', 'yellow', 'red']
}

Map = geemap.Map()
Map.centerObject(aoi, 13)
Map.addLayer(lst_2013.clip(aoi), visualization_params, 'LST 2013 (lub 2014)')
Map.addLayer(lst_2023.clip(aoi), visualization_params, 'LST 2023')
Map.addLayer(corine.clip(aoi), {}, 'CORINE')
Map.addLayer(emissivity.clip(aoi), {'min': 0.9, 'max': 1.0}, 'Emissivity')
Map.save("LST_Comparison_2013_2023.html")
webbrowser.open("LST_Comparison_2013_2023.html")



True

## Step 7: (Optional) Analyze statistics by land cover class

In [37]:
print("\nŚrednia LST w klasach CORINE (2013 i 2023):")
for corine_class in corine_classes:
    zone_mask = corine.eq(corine_class)

    masked_lst_2013 = lst_2013.updateMask(zone_mask)
    masked_lst_2023 = lst_2023.updateMask(zone_mask)

    stats_2013 = masked_lst_2013.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=100,
        maxPixels=1e9
    )

    stats_2023 = masked_lst_2023.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=100,
        maxPixels=1e9
    )

    try:
        mean_2013 = stats_2013.getInfo().get('LST')
        mean_2023 = stats_2023.getInfo().get('LST')
        print(f'CORINE {corine_class}: {mean_2013:.2f} K (2013) → {mean_2023:.2f} K (2023)')
    except Exception as e:
        print(f'Błąd dla klasy {corine_class}: {e}')



Średnia LST w klasach CORINE (2013 i 2023):
Błąd dla klasy 111: unsupported format string passed to NoneType.__format__
CORINE 112: 315.75 K (2013) → 310.89 K (2023)
Błąd dla klasy 121: unsupported format string passed to NoneType.__format__
Błąd dla klasy 211: unsupported format string passed to NoneType.__format__
Błąd dla klasy 311: unsupported format string passed to NoneType.__format__
Błąd dla klasy 412: unsupported format string passed to NoneType.__format__
Błąd dla klasy 324: unsupported format string passed to NoneType.__format__
CORINE 231: 307.23 K (2013) → 303.74 K (2023)


## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade